Using EEG data from CHB and creating a custom OOP Python program using a wide-range of higher mathematics involving Linear Algebra and Non-Linear Dynamics, I created proprietary model to provide the likelihood that the EEG readings from any patient at any moment in time can accurately say if the patient is in a seizure or not.

In [14]:
file_model_train = '/Users/jshensley/Desktop/Cap3_Nico_MUSC/seizure_algo/dataframes/model_df.csv'
model_training_df = pd.read_csv(file_model_train)
model_training_df['seizure'] = 0
model_training_df['seizure'].iloc[10239:] = 1

loaded_h5_model = tf.keras.models.load_model("best_model_HDF5_format.h5")

# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------
file_18 = '/Users/jshensley/Desktop/Cap3_Nico_MUSC/seizure_algo/dataframes/model18_df.csv'
df_18 = pd.read_csv(file_18)
df_18.drop(['Unnamed: 0'], axis=1, inplace=True)
df_18.drop([20478], inplace=True)
df_18.fillna(method='ffill', inplace=True)

predict_18_nump = df_18.to_numpy()
predict_18_nump = predict_18_nump[None, :, :]
predict_18 = loaded_h5_model.predict(predict_18_nump)
predict_18 = np.squeeze(predict_18)

predict_18_series = pd.Series(predict_18, name='predict_18')
predict_18_series.drop([0], inplace=True)


# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------
file_vns = '/Users/jshensley/Desktop/Cap3_Nico_MUSC/seizure_algo/dataframes/model_vns_df.csv'
df_vns = pd.read_csv(file_vns)
df_vns.drop('Unnamed: 0', axis=1, inplace=True)
df_vns.drop([20478], inplace=True)
df_vns.fillna(method='ffill', inplace=True)

predict_vns_nump = df_vns.to_numpy()
predict_vns_nump = predict_vns_nump[None, :, :]
predict_vns = loaded_h5_model.predict(predict_vns_nump)
predict_vns = np.squeeze(predict_vns)

predict_vns_series = pd.Series(predict_vns, name='predict_vns')
predict_vns_series.drop([0], inplace=True)
accuracy = pd.concat([predict_18_series, predict_vns_series, model_training_df['seizure']], axis=1)

/var/folders/lc/nbgy6kvs4bzdy9cwsd9m4f600000gn/T/ipykernel_6347/2684665916.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_training_df['seizure'].iloc[10239:] = 1


1/1 [==============================] - 0s 202ms/step


In [16]:
accuracy

,predict_18,predict_vns,seizure
1,0.007238,0.001046,0
2,0.007056,0.001043,0
3,0.006935,0.001033,0
4,0.007337,0.001095,0
5,0.007025,0.001134,0
...,...,...,...
20474,0.992988,0.998941,1
20475,0.992786,0.998936,1
20476,0.992934,0.998986,1
20477,0.992649,0.998928,1


The first two columns are two different patients. 'Predict-18 is a female and 'Predict_vns,' also a female, but has a VNS surgically inserted.

The original dataframe have been concatenated so that it contains the 20 seconds before the seizure, the seizure start at line 10239, and the approximately 20 seconds after the seizure. In other words, the seizure start lines up on axis=1 for ease in visualization.

THe third column shows where the non-seizure zero rows are and where the seizure one rows are.

The Tensorflow model has been trained on a another dataset from the same patient as 'Predict_18' but a year and a half earlier in life.

The model was run in a separate .py program and saved as a .h5 file.

The 'Predict-18' and 'Predict_vns' dataframes were not seen by the model.

The model took almost an hour to train running on a Macbook Pro 2020 M1, ended with a loss closing in on a Plank distance of zero and predicts the new data with 100& accuracy.

The original goal was a high-precision model to eliminate false positives.

False positives are to be avoided at all costs to avoid real time problems in a hospital scenario.
